In [19]:
import csv
import pandas as pd

# Crawled cafe data (instagram)
df_=pd.read_csv("jeju_cafe.csv")
df_1=pd.read_csv("jeju_cafe1.csv")
df_2=pd.read_csv("jeju_cafe2.csv")
df_3=pd.concat([df_,df_1,df_2])
df_fin=df_3.groupby("name",as_index=False).sum().sort_values(by='mention',ascending=False)
del df_fin['type']
lst = df_fin.values.tolist()
print("Result 0")
print(f"Not Found: {len(lst):>3} rows")

#File 1 (소상공인시장진흥공단_상가(상권)정보 제주)
df_file1=pd.read_csv("jeju_final.csv",usecols=['상호명','도로명주소','위도','경도'])
file1=df_file1.values.tolist()      

for x in lst:
    cnt = 0
    for item in file1:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            x.append(item[1])
            x.append(item[3])
            x.append(item[2])
    if cnt == 0:
        x.append('')

cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("-------------------")
print("Result 1")
print(f"   File 1: {cnt1:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")
df1 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df1

Result 0
Not Found: 354 rows
-------------------
Result 1
   File 1:  21 rows
Not Found: 333 rows
    Total: 354 rows


,name,mention,address,latitude,longitude
0,카페더라이트,52,제주특별자치도 서귀포시 성산읍 한도로 267,33.466171,126.935974
1,새빌,24,제주특별자치도 서귀포시 대정읍 노을해안로 216-30,33.246273,126.207174
2,제주카페,14,,NaN,NaN
3,브라보비치,11,제주특별자치도 서귀포시 성산읍 해맞이해안로 2614,33.477598,126.909302
4,웨이뷰,9,,NaN,NaN
...,...,...,...,...,...
349,도리미노노,1,,NaN,NaN
350,델리 미트 제주,1,,NaN,NaN
351,더블더블,1,,NaN,NaN
352,더 크로플리,1,,NaN,NaN


In [20]:
#File 2 (제주 market)
file2 = []
with open('jeju_market.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        file2.append([row[0], row[12], row[15], row[14]])

f1_cnt = 0
f2_cnt = 0
for x in lst:
    if len(x) == 5:
        f1_cnt += 1
        continue
    cnt = 0
    for item in file2:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            f2_cnt += 1
            x[2] = item[1]
            x.append(item[2])
            x.append(item[3])

cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 2")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")

df2 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df2

Result 2
   File 1:  21 rows
   File 2:   1 rows
Not Found: 332 rows
    Total: 354 rows


,name,mention,address,latitude,longitude
0,카페더라이트,52,제주특별자치도 서귀포시 성산읍 한도로 267,33.466171,126.935974
1,새빌,24,제주특별자치도 서귀포시 대정읍 노을해안로 216-30,33.246273,126.207174
2,제주카페,14,,None,None
3,브라보비치,11,제주특별자치도 서귀포시 성산읍 해맞이해안로 2614,33.477598,126.909302
4,웨이뷰,9,,None,None
...,...,...,...,...,...
349,도리미노노,1,,None,None
350,델리 미트 제주,1,,None,None
351,더블더블,1,,None,None
352,더 크로플리,1,,None,None


In [21]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    if geo is not None:
        crd = [geo.latitude, geo.longitude]
        return crd
    else:
        return None

def addr_revise(address):
    str_del1 = ','
    str_del2 = '('
    if str_del1 in address:
        address = address[:address.index(str_del1)]
    if str_del2 in address:
        address = address[:address.index(str_del2)]
    return address

In [22]:
#File 3 (제주데이터허브)
file3 = []
with open('jeju_food.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        if row[5].find('폐업') == -1:
            file3.append([row[0], row[13]])

f3_cnt = 0
for x in lst:
    if len(x) == 5:
        continue
    cnt = 0
    for item in file3:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            addr = addr_revise(item[1])
            x[2] = addr
            if geocoding(addr) is not None:
                x.append(geocoding(addr)[0])
                x.append(geocoding(addr)[1])
                f3_cnt += 1
cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 3")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"   File 3: {f3_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")        
df3 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df3

Result 3
   File 1:  21 rows
   File 2:   1 rows
   File 3:   2 rows
Not Found: 330 rows
    Total: 354 rows


,name,mention,address,latitude,longitude
0,카페더라이트,52,제주특별자치도 서귀포시 성산읍 한도로 267,33.466171,126.935974
1,새빌,24,제주특별자치도 서귀포시 대정읍 노을해안로 216-30,33.246273,126.207174
2,제주카페,14,,None,None
3,브라보비치,11,제주특별자치도 서귀포시 성산읍 해맞이해안로 2614,33.477598,126.909302
4,웨이뷰,9,,None,None
...,...,...,...,...,...
349,도리미노노,1,,None,None
350,델리 미트 제주,1,,None,None
351,더블더블,1,,None,None
352,더 크로플리,1,,None,None


In [23]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import time

import re

driver = webdriver.Chrome()
driver.implicitly_wait(3)

def switch_frame(frame):
    driver.switch_to.default_content()
    driver.switch_to.frame(frame)

def remove_after_number(str):
    result = re.sub(r'(\d)\s.*', r'\1', str)
    return result

crawl = []
for e in lst:
    if len(e) == 3:
        crawl.append([lst.index(e),e[0]])
crawl_data = []
f4_cnt = 0
for r in range(len(crawl)):
    driver.get('https://map.naver.com/')
    time.sleep(3)
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(crawl[r][1])
    search.send_keys(Keys.ENTER)

    time.sleep(1)
    switch_frame('searchIframe')
    time.sleep(3)

    res_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
    print(f'{len(res_list)}개의 검색 결과가 나왔습니다.')

    for data in range(len(res_list)):
        if len(res_list) >= 10:
            break
        if len(res_list) == 1:
            switch_frame('entryIframe')
            addr_button = driver.find_elements(By.CSS_SELECTOR, 'div.vV_z_ > a')
            addr_button.__getitem__(0).click()
            time.sleep(2)
            name_path = '/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[1]'
            name = driver.find_element(By.XPATH, name_path).text
            addr = driver.find_elements(By.CSS_SELECTOR, 'div.vV_z_ > div.Y31Sf > div.nQ7Lh')
            road = addr.__getitem__(0).text
            road_addr = road[3:(len(road) - 2)]
            if road_addr.startswith('제주'):
                real_addr = remove_after_number(road_addr)
                lst[crawl[r][0]][2] = real_addr
                if geocoding(real_addr) is not None:
                    lst[crawl[r][0]].append(geocoding(real_addr)[0])
                    lst[crawl[r][0]].append(geocoding(real_addr)[1])
                    print(f"Crawl: {name}, Name: {crawl[r][1]}")
                    lst[crawl[r][0]][0] = name
                    print(lst[crawl[r][0]])
                    f4_cnt += 1
        else:
            name_ = driver.find_elements(By.CSS_SELECTOR, 'span.YwYLL')
            name = name_.__getitem__(data).text
            addr_button = driver.find_elements(By.CSS_SELECTOR, 'span.lWwyx > a')
            addr_button.__getitem__(data).click()
            time.sleep(2)
            addr = driver.find_elements(By.CSS_SELECTOR, 'div > div.AbTyi > div.zZfO1')
            road = addr.__getitem__(0).text
            road_addr = road[3:(len(road) - 2)]
            if road_addr.startswith('제주'):
                real_addr = remove_after_number(road_addr)
                lst[crawl[r][0]][2] = real_addr
                if geocoding(real_addr) is not None:
                    lst[crawl[r][0]].append(geocoding(real_addr)[0])
                    lst[crawl[r][0]].append(geocoding(real_addr)[1])
                    print(f"Crawl: {name}, Name: {crawl[r][1]}")
                    lst[crawl[r][0]][0] = name
                    print(lst[crawl[r][0]])
                    f4_cnt += 1
                    break
            addr_button.__getitem__(data).click()

        time.sleep(5)

df4 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df4


0개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
Crawl: 웨이뷰 협재점, Name: 웨이뷰
['웨이뷰 협재점', 9, '제주 제주시 한림읍 옹포7길 25-3', 33.4069704, 126.254288]
1개의 검색 결과가 나왔습니다.
Crawl: 마음에온, Name: 마음에온 카페
['마음에온', 8, '제주 제주시 칠성로길 29-1', 33.5144683, 126.5262182]
2개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
9개의 검색 결과가 나왔습니다.
Crawl: 카페마음마루, Name: 제주카페마루
['카페마음마루', 2, '제주 제주시 아란14길 3', 33.4711333, 126.5441525]
1개의 검색 결과가 나왔습니다.
10개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
4개의 검색 결과가 나왔습니다.
Crawl: 웨스트그라운드, Name: 제주카페그라운드
['웨스트그라운드', 1, '제주 제주시 한경면 신한로 183-16', 33.347143, 126.1897426]
6개의 검색 결과가 나왔습니다.
Crawl: 파이굽는 돌담집, Name: 제주카페돛담
['파이굽는 돌담집', 1, '제주 제주시 아란9길 48', 33.4717481, 126.5406229]
10개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
5개의 검색 결과가 나왔습니다.
8개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
10개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
10개의 검색 결과가 나왔습니다.
10개의 검색 결과가 나왔습니다.
3개의 검색 결과가 나왔습니다.
Crawl: 르브아 커피, Name: 제주동백수목원 카페

,name,mention,address,latitude,longitude
0,카페더라이트,52,제주특별자치도 서귀포시 성산읍 한도로 267,33.466171,126.935974
1,새빌,24,제주특별자치도 서귀포시 대정읍 노을해안로 216-30,33.246273,126.207174
2,제주카페,14,,None,None
3,브라보비치,11,제주특별자치도 서귀포시 성산읍 해맞이해안로 2614,33.477598,126.909302
4,웨이뷰 협재점,9,제주 제주시 한림읍 옹포7길 25-3,33.40697,126.254288
...,...,...,...,...,...
349,도리미노노,1,,None,None
350,델리 미트 제주,1,,None,None
351,더블더블,1,,None,None
352,더 크로플리,1,,None,None


In [24]:
cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 4")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"   File 3: {f3_cnt:>3} rows")
print(f"    Crawl: {f4_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")

final_lst = []
for i in lst:
    final_lst.append(i)
print(final_lst)

Result 4
   File 1:  21 rows
   File 2:   1 rows
   File 3:   2 rows
    Crawl:  11 rows
Not Found: 319 rows
    Total: 354 rows
[['카페더라이트', 52, '제주특별자치도 서귀포시 성산읍 한도로 267', 33.4661713753033, 126.935973729891], ['새빌', 24, '제주특별자치도 서귀포시 대정읍 노을해안로 216-30', 33.2462728003026, 126.207174417376], ['제주카페', 14, ''], ['브라보비치', 11, '제주특별자치도 서귀포시 성산읍 해맞이해안로 2614', 33.4775976523995, 126.909301990313], ['웨이뷰 협재점', 9, '제주 제주시 한림읍 옹포7길 25-3', 33.4069704, 126.254288], ['마음에온', 8, '제주 제주시 칠성로길 29-1', 33.5144683, 126.5262182], ['리그램', 6, ''], ['쉬어갓', 5, '제주특별자치도 서귀포시 성산읍 일주동로 5023', 33.3860851013057, 126.882329800264], ['위이', 5, '제주특별자치도 서귀포시 안덕면 신화역사로682번길 12', 33.3127519347285, 126.359105799759], ['지구상회', 5, '제주특별자치도 제주시 애월읍 애월해안로 715', 33.4840001976835, 126.377379932118], ['하라케케', 4, '제주특별자치도 서귀포시 속골로 29-10', 33.2428543200944, 126.527782166788], ['프릳츠 제주성산', 4, '제주 서귀포시 성산읍 일출로 222'], ['쿠리킨톤', 3, ''], ['커피템플', 2, '제주특별자치도 제주시 영평길 269', 33.4781538312649, 126.57030222334], ['카페마음마루', 2, '제주 제주시 아란14길 3'

In [39]:
print(final_lst)

[['카페더라이트', 52, '제주특별자치도 서귀포시 성산읍 한도로 267', 33.4661713753033, 126.935973729891], ['새빌', 24, '제주특별자치도 서귀포시 대정읍 노을해안로 216-30', 33.2462728003026, 126.207174417376], ['제주카페', 14, ''], ['브라보비치', 11, '제주특별자치도 서귀포시 성산읍 해맞이해안로 2614', 33.4775976523995, 126.909301990313], ['웨이뷰 협재점', 9, '제주 제주시 한림읍 옹포7길 25-3', 33.4069704, 126.254288], ['마음에온', 8, '제주 제주시 칠성로길 29-1', 33.5144683, 126.5262182], ['리그램', 6, ''], ['쉬어갓', 5, '제주특별자치도 서귀포시 성산읍 일주동로 5023', 33.3860851013057, 126.882329800264], ['위이', 5, '제주특별자치도 서귀포시 안덕면 신화역사로682번길 12', 33.3127519347285, 126.359105799759], ['지구상회', 5, '제주특별자치도 제주시 애월읍 애월해안로 715', 33.4840001976835, 126.377379932118], ['하라케케', 4, '제주특별자치도 서귀포시 속골로 29-10', 33.2428543200944, 126.527782166788], ['프릳츠 제주성산', 4, '제주 서귀포시 성산읍 일출로 222'], ['쿠리킨톤', 3, ''], ['커피템플', 2, '제주특별자치도 제주시 영평길 269', 33.4781538312649, 126.57030222334], ['카페마음마루', 2, '제주 제주시 아란14길 3', 33.4711333, 126.5441525], ['어린왕자감귤밭', 2, '제주특별자치도 서귀포시 대정읍 추사로36번길 45-1', 33.2532868801428, 126.281885810743], ['Wiee', 2, '제주 

In [40]:
final_result = []
final_result.append(['name', 'mention', 'address', 'latitude', 'longitude', 'type'])
for e in final_lst:
    if len(e) == 5:
        e.append(0)
        print(e)
        final_result.append(e)
print(len(final_result))

with open('output2.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(final_result)

['카페더라이트', 52, '제주특별자치도 서귀포시 성산읍 한도로 267', 33.4661713753033, 126.935973729891, 0]
['새빌', 24, '제주특별자치도 서귀포시 대정읍 노을해안로 216-30', 33.2462728003026, 126.207174417376, 0]
['브라보비치', 11, '제주특별자치도 서귀포시 성산읍 해맞이해안로 2614', 33.4775976523995, 126.909301990313, 0]
['웨이뷰 협재점', 9, '제주 제주시 한림읍 옹포7길 25-3', 33.4069704, 126.254288, 0]
['마음에온', 8, '제주 제주시 칠성로길 29-1', 33.5144683, 126.5262182, 0]
['쉬어갓', 5, '제주특별자치도 서귀포시 성산읍 일주동로 5023', 33.3860851013057, 126.882329800264, 0]
['위이', 5, '제주특별자치도 서귀포시 안덕면 신화역사로682번길 12', 33.3127519347285, 126.359105799759, 0]
['지구상회', 5, '제주특별자치도 제주시 애월읍 애월해안로 715', 33.4840001976835, 126.377379932118, 0]
['하라케케', 4, '제주특별자치도 서귀포시 속골로 29-10', 33.2428543200944, 126.527782166788, 0]
['커피템플', 2, '제주특별자치도 제주시 영평길 269', 33.4781538312649, 126.57030222334, 0]
['카페마음마루', 2, '제주 제주시 아란14길 3', 33.4711333, 126.5441525, 0]
['어린왕자감귤밭', 2, '제주특별자치도 서귀포시 대정읍 추사로36번길 45-1', 33.2532868801428, 126.281885810743, 0]
['한라산', 2, '제주특별자치도 서귀포시 표선면 중산간동로 4669', 33.3890832377531, 126.797005141356, 0]
['웨